In [1]:
os.chdir("/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/")
result_dir = "../by_tissue/"

In [3]:
def process_tissue(file_list):
    # file_list = ["stomach", ["sto_IP_4.txt",'sto_IP_5.txt','CRR042306.txt','CRR042308.txt']]
    result_file = os.path.join(result_dir, "%s.txt" % file_list[0])
    file_list = file_list[1]
    df = pd.read_table(file_list[0])
    df['effect_%s'% file_list[0].split(".")[0]] = df['foldchange']
    for x in file_list[1:]:
        df_i = pd.read_table(x)
        df_i['effect_%s'% x.split(".")[0]] = df_i['foldchange']
        df = df.merge(df_i, on=['contig','position','refAllele','altAllele'], how="outer")
    selected_cols = ['contig','position','refAllele','altAllele'] + [x for x in df.columns if "effect_" in x]
    df_result = df[selected_cols]
    df_result.to_csv(result_file, sep="\t", index=False)

In [4]:
file_1 = ["stomach", ["sto_IP_4.txt",'sto_IP_5.txt','CRR042306.txt','CRR042308.txt']]
file_2 = ["muscle", ["mus_IP_4.txt",'mus_IP_5.txt','CRR042298.txt','CRR055549.txt']]
file_3 = ["lung", ["lung_IP_4.txt",'lung_IP_5.txt','CRR073020.txt','CRR042296.txt','CRR073018.txt','CRR055533.txt']]
# file_4 = ["liver", ["liver_IP_1.txt",'liver_IP_2.txt','liver_IP_3.txt','CRR042294.txt']]
file_5 = ["heart", ["heart_IP_1.txt",'heart_IP_2.txt','heart_IP_3.txt','CRR055527.txt','CRR042290.txt']]
file_6 = ["brain", ["brain_IP_1.txt",'brain_IP_2.txt','brain_IP_3.txt','CRR042286.txt','CRR042282.txt','CRR042284.txt','CRR042292.txt','CRR073016.txt','CRR055553.txt']]
for file_list in [file_1,file_2,file_3,file_5,file_6]:
    process_tissue(file_list)

In [27]:
### process each tissue
os.chdir("/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/by_tissue/")
txt_list = glob.glob("*.txt")
for txt in txt_list:
    tissue = txt.split(".")[0]
    df = pd.read_table(txt)
    df['term'] = df['contig'] + ":" + df['position'].astype(str) + ":" + df['refAllele'] + ":" + df['altAllele']
    fetal_cols, adult_cols = [x for x in df.columns if "IP_" in x], [x for x in df.columns if "_CRR" in x]
    df_fetal, df_adult = df[['term'] + fetal_cols], df[['term'] + adult_cols]
    df_fetal, df_adult = select_same_trend(df_fetal), select_same_trend(df_adult)
    convert_to_bed(df_fetal, tissue, "fetal")
    convert_to_bed(df_adult, tissue, "adult")
    fetal_mt_list, fetal_lt_list= mark_snp(df_fetal)
    adult_mt_list, adult_lt_list= mark_snp(df_adult)
    
    common_list, diff_list = [],[]
    for x in fetal_mt_list:
        if x in adult_mt_list:
            common_list.append(x)
        elif x in adult_lt_list:
            diff_list.append(x)
    for x in fetal_lt_list:
        if x in adult_lt_list:
            common_list.append(x)
        elif x in adult_mt_list:
            diff_list.append(x)
    list_to_bed(common_list, tissue, "same")
    list_to_bed(diff_list, tissue, "opposite")

In [26]:
def select_same_trend(df):
    array_list = []
    df = df.dropna(thresh=3)
    cols = df.columns
    for i, v in df.iterrows():
        values = v.dropna()[1:]
        if (min(values) >= 1) or (max(values) < 1):
            array_list.append(v)
    df_result = pd.DataFrame(array_list)
    df_result.columns = cols
    return df_result

def convert_to_bed(df, tissue, data_type):
    df['chr'] = df['term'].str.split(":").str[0]
    df['pos'] = df['term'].str.split(":").str[1]
    df_bed = df[['chr','pos','pos','term']]
    df_bed.to_csv("%s_%s.bed"%(tissue,data_type), sep="\t", header=False, index=False)
    
def mark_snp(df):
    mt_list, lt_list = [], []
    s_cols = [x for x in df.columns if "effect_" in x]
    for i,j in df.iterrows():
        values = [j[x] for x in s_cols if not np.isnan(j[x])]
        if np.mean(values) > 1:
            mt_list.append(j['term'])
        else:
            lt_list.append(j['term'])
    return mt_list, lt_list

def list_to_bed(in_list, tissue, oriention):
    result_file = "%s_%s.bed" % (tissue, oriention)
    if len(in_list) == 0:
        os.system("touch %s" % result_file)
    else:
        array_list = []
        for x in in_list:
            array_list.append(x.split(":"))
        df = pd.DataFrame(array_list)
        df.columns = ['contig','position','refAllele','altAllele']
        df_sub = df[['contig','position','position']]
        df_sub.to_csv(result_file, sep="\t", header=False, index=False)

In [3]:
# select different loci based on overlap;
os.chdir("/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/by_tissue/")
tissue_list = ['brain','heart','stomach','lung','muscle']
for tissue in tissue_list:
    fetal, adult = "%s_fetal.bed" % tissue, "%s_adult.bed" % tissue
    res_fetal, res_adult = "%s_diff_fetal"%tissue, "%s_diff_adult"%tissue
    with open(fetal, 'r') as f_fetal:
        fetal_lines = f_fetal.readlines()
    with open(adult, 'r') as f_adult:
        adult_lines = f_adult.readlines()
    with open(res_fetal, 'w') as fw_fetal:
        fw_fetal.writelines([x for x in fetal_lines if x not in adult_lines])
    with open(res_adult, 'w') as fw_adult:
        fw_adult.writelines([x for x in adult_lines if x not in fetal_lines])